<a href="https://www.kaggle.com/code/mainguynhuynh/rice-yolov12-multiscale?scriptVersionId=298203215" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# 1. Clone repository về
!git clone https://github.com/sunsmarterjie/yolov12.git
%cd yolov12

# 2. Cài đặt các thư viện cần thiết
# Lưu ý: Flash Attention yêu cầu GPU phù hợp, trên Kaggle bạn có thể cài đặt bản build sẵn
!pip install -r requirements.txt
!pip install -e .

# 3. Cài đặt thêm thư viện ultralytics nếu cần (vì YOLOv12 dựa trên framework này)
!pip install ultralytics

# ── Tạo folder output (Kaggle lưu files ở đây) ──
import os
OUTPUT_DIR = '/kaggle/working/outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Output folder: {OUTPUT_DIR}")

In [ ]:
pip install roboflow

In [ ]:
from roboflow import Roboflow
import os

rf      = Roboflow(api_key="hr9lzB96MhmoITXPgPkI")
project = rf.workspace("tungro-tqtks").project("rice-disease-rwuhc")
version = project.version(3)
dataset = version.download("yolov12", location="/kaggle/working/yolov12/rice-disease-3")

dataset_path   = "/kaggle/working/yolov12/rice-disease-3"
data_yaml_path = os.path.join(dataset_path, "data.yaml")

print(f"✅ Dataset: {dataset_path}")
print(f"✅ YAML:    {data_yaml_path}")

In [ ]:
import os, yaml

LABEL_MAPPING = {
    'rice': 0,
    'blas': 1, 'Rice Blast Disease': 1, 'rice_blast_disease': 1,
    'Rice-Blast-Disease': 1, 'Rice Blast Disease-': 1,
    'brown_spot_disease': 2, 'bercak_coklat': 2,
    'Rice_Brown_spot': 2, 'Brown Spot Disease': 2,
    'narrow_brown_spot_disease': 3, 'Narrow Brown Spot Disease': 3,
    'hawar_daun_bakteri': 4, 'Rice_Bacterial_Blight': 4,
    'dirty_panicle_disease': 5, 'Dirty Panicle Disease': 5,
    'tungro': 6
}
NEW_NAMES = ['Rice', 'Blast', 'Brown_Spot', 'Narrow_Brown_Spot',
             'Bacterial_Blight', 'Dirty_Panicle', 'Tungro']

with open(data_yaml_path, 'r') as f:
    old_data  = yaml.safe_load(f)
    old_names = old_data['names']

def clean_labels(folder_path):
    for fname in [f for f in os.listdir(folder_path) if f.endswith('.txt')]:
        fpath     = os.path.join(folder_path, fname)
        new_lines = []
        with open(fpath, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if not parts: continue
            old_id = int(parts[0])
            if old_id < len(old_names):
                old_name = old_names[old_id]
                if old_name in LABEL_MAPPING:
                    new_id = LABEL_MAPPING[old_name]
                    new_lines.append(f"{new_id} {' '.join(parts[1:])}\n")
        with open(fpath, 'w') as f:
            f.writelines(new_lines)

for split in ['train', 'valid', 'test']:
    lbl_dir = os.path.join(dataset_path, split, 'labels')
    if os.path.exists(lbl_dir):
        print(f"  Đang xử lý {split}...")
        clean_labels(lbl_dir)

old_data['names'] = NEW_NAMES
old_data['nc']    = 7
old_data['train'] = 'train/images'
old_data['val']   = 'valid/images'
old_data['test']  = 'test/images'

with open(data_yaml_path, 'w') as f:
    yaml.dump(old_data, f)

print(f"✅ Chuẩn hóa xong: {NEW_NAMES}")

In [ ]:
# ═══════════════════════════════════════════════════════
# CELL 4: PATCH MULTI-SCALE BiFPN
# ═══════════════════════════════════════════════════════

import re

target_path = '/kaggle/working/yolov12/ultralytics/nn/tasks.py'

bifpn_code = """
# [CUSTOM BLOCK START] -------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F

class Concat_BiFPN(nn.Module):
    def __init__(self, c1, dimension=1):
        super().__init__()
        self.d = dimension
        combined_channels = sum(c1) if isinstance(c1, list) else c1

        self.compress = nn.Sequential(
            nn.Conv2d(combined_channels, combined_channels // 4, kernel_size=1),
            nn.BatchNorm2d(combined_channels // 4),
            nn.ReLU(inplace=True)
        )
        self.global_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(combined_channels // 4, combined_channels // 16, 1),
            nn.ReLU(inplace=True)
        )
        self.local_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(combined_channels // 4, combined_channels // 16, 1),
            nn.ReLU(inplace=True)
        )
        self.medium_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=1, padding=2),
            nn.Conv2d(combined_channels // 4, combined_channels // 16, 1),
            nn.ReLU(inplace=True)
        )
        fusion_channels = 3 * (combined_channels // 16)
        self.fusion = nn.Sequential(
            nn.Conv2d(fusion_channels, combined_channels // 8, 1),
            nn.ReLU(inplace=True)
        )
        self.weight_predictor = nn.Sequential(
            nn.Conv2d(combined_channels // 8, 2, kernel_size=1),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        if not isinstance(x, list) or len(x) < 2:
            return torch.cat(x, self.d) if isinstance(x, list) else x
        combined   = torch.cat(x, dim=1)
        compressed = self.compress(combined)
        global_feat = self.global_pool(compressed)
        global_feat = F.interpolate(global_feat, size=compressed.shape[2:], mode='nearest')
        local_feat  = self.local_pool(compressed)
        medium_feat = self.medium_pool(compressed)
        multi_scale = torch.cat([global_feat, local_feat, medium_feat], dim=1)
        fused       = self.fusion(multi_scale)
        weights     = self.weight_predictor(fused)
        w1 = weights[:, 0:1, :, :]
        w2 = weights[:, 1:2, :, :]
        return torch.cat([x[0] * w1, x[1] * w2], self.d)
# [CUSTOM BLOCK END] ---------------------------
"""

with open(target_path, 'r') as f:
    content = f.read()

# Xóa version cũ nếu có
if "class Concat_BiFPN" in content:
    content = re.sub(
        r'# \[CUSTOM BLOCK START\].*?# \[CUSTOM BLOCK END\][^\n]*\n',
        '', content, flags=re.DOTALL
    )
    print("⚠️  Đã xóa BiFPN cũ")

# Chèn mới
before, after = content.split("def parse_model(", 1)
content = before + bifpn_code + "\ndef parse_model(" + after

with open(target_path, 'w') as f:
    f.write(content)

print("✅ Multi-Scale BiFPN patched!")

In [ ]:
# ═══════════════════════════════════════════════════════
# CELL 5: FIX THAM SỐ C1
# ═══════════════════════════════════════════════════════

with open(target_path, 'r') as f:
    lines = f.readlines()

new_lines, found_fix = [], False

for line in lines:
    new_lines.append(line)
    if "c2 = sum(ch[x] for x in f)" in line:
        indent   = line[:line.find('c2')]
        fix_code = f"{indent}if m is Concat_BiFPN:\n{indent}    args.insert(0, c2)\n"
        if "if m is Concat_BiFPN:" not in "".join(lines):
            new_lines.append(fix_code)
            found_fix = True

if found_fix:
    with open(target_path, 'w') as f:
        f.writelines(new_lines)
    print("✅ Fix c2 args xong!")

with open(target_path, 'r') as f:
    content = f.read()

if "if m is Concat:" in content:
    content = content.replace("if m is Concat:", "if m in (Concat, Concat_BiFPN):")
    with open(target_path, 'w') as f:
        f.write(content)
    print("✅ Fix Concat condition xong!")


In [ ]:
# ═══════════════════════════════════════════════════════
# CELL 6: TẠO YAML BiFPN
# ═══════════════════════════════════════════════════════

original_yaml = '/kaggle/working/yolov12/ultralytics/cfg/models/v12/yolov12.yaml'
custom_yaml   = '/kaggle/working/yolov12_bifpn.yaml'

with open(original_yaml, 'r') as f:
    content = f.read()

content = content.replace('Concat', 'Concat_BiFPN')

with open(custom_yaml, 'w') as f:
    f.write(content)

print(f"✅ Tạo YAML: {custom_yaml}")


In [ ]:
# import os
# from ultralytics import YOLO


# # --- 2. Tải weights pre-trained  ---
# if not os.path.exists('yolov12n.pt'):
#     print("Đang tải weights yolov12n.pt...")
#     !wget -q https://github.com/syp2ysy/yolov12/releases/download/v1.0/yolov12n.pt

# # --- 3. Khởi tạo Model ---
# print("Đang khởi tạo model với kiến trúc BiFPN...")
# # Lưu ý: File yaml nằm ở /kaggle/working/yolov12_bifpn.yaml
# model = YOLO('/kaggle/working/yolov12_bifpn.yaml')

# # --- 4. Load Weights (Transfer Learning) ---
# try:
#     # Chỉ load những layer khớp (Backbone, Head), bỏ qua BiFPN (chưa có weight)
#     model.load('yolov12n.pt')
#     print("Đã load weights pre-trained thành công")
# except Exception as e:
#     print(f"Cảnh báo load weights: {e}")
#     print("Model sẽ train từ đầu (scratch) hoặc chỉ load một phần.")

In [ ]:
# # --- 5. Bắt đầu Train ---
# results = model.train(
#     data='/kaggle/working/yolov12/rice-disease-3/data.yaml',
#     epochs=100,       
#     imgsz=640,             
#     batch=16,              
#     project='runs/train', 
#     name='yolov12_bifpn_rice_default' 
# )

In [ ]:
# ═══════════════════════════════════════════════════════
# CELL 7: TRAIN
# ── Output lưu vào /kaggle/working/outputs/ ──
# ── Kaggle tự động lưu /kaggle/working/ vào Files tab ──
# ═══════════════════════════════════════════════════════

import sys
[sys.modules.pop(k) for k in list(sys.modules) if 'ultralytics' in k]

from ultralytics import YOLO

# ── Load pretrained ──
weights_path = '/kaggle/input/models/mainguynhuynh/finetune-1-dynamic/other/default/1/dynamic_finetune1.pt'

if os.path.exists(weights_path):
    print(f"🔄 Fine-tune từ: {weights_path}")
    model = YOLO(weights_path)
else:
    print("🔄 Train từ YAML + pretrained backbone")
    model = YOLO(custom_yaml)
    model.load('yolov12n.pt')

# ── Train ──
# KEY: project trỏ vào /kaggle/working/outputs/
# → Kaggle tự động lưu vào Files tab!
results = model.train(
    data        = data_yaml_path,
    epochs      = 100,
    imgsz       = 640,
    batch       = 16,
    lr0         = 0.005,
    workers     = 2,
    patience    = 20,
    save_period = 10,    # Lưu checkpoint mỗi 10 epoch

    # ← QUAN TRỌNG: Lưu vào /kaggle/working/outputs/
    project     = '/kaggle/working/outputs',
    name        = 'yolov12_multiscale_bifpn',
    exist_ok    = False,
)

mAP50 = results.results_dict.get('metrics/mAP50(B)', 0)
print(f"\n✅ mAP@0.5 = {mAP50*100:.1f}%")
print(f"💾 Weights: /kaggle/working/outputs/yolov12_multiscale_bifpn/weights/")


In [ ]:
# ═══ EVAL CELL - Chạy sau khi train xong ═══
import sys, os
[sys.modules.pop(k) for k in list(sys.modules) if 'ultralytics' in k]

# Patch BiFPN Multi-Scale (BẮT BUỘC cho YOLOv12!)
target_path = '/kaggle/working/yolov12/ultralytics/nn/tasks.py'

# Verify patch còn đó không
with open(target_path) as f:
    content = f.read()
print("BiFPN:", "class Concat_BiFPN" in content)
print("c2 fix:", "if m is Concat_BiFPN:" in content)

from ultralytics import YOLO

best_pt = '/kaggle/working/outputs/yolov12_multiscale_bifpn/weights/best.pt'

# Nếu chưa có best.pt thì dùng last.pt
if not os.path.exists(best_pt):
    best_pt = '/kaggle/working/outputs/yolov12_multiscale_bifpn/weights/last.pt'

print(f"\n✅ Loading: {best_pt}")
model = YOLO(best_pt)

metrics = model.val(
    data  = '/kaggle/working/yolov12/rice-disease-3/data.yaml',
    split = 'test',
    imgsz = 640,
    batch = 16,
)

print("\n" + "="*45)
print(f"PRECISION : {metrics.box.mp:.4f}")
print(f"RECALL    : {metrics.box.mr:.4f}")
print(f"mAP@50    : {metrics.box.map50:.4f}")
print(f"mAP@50-95 : {metrics.box.map:.4f}")
print("="*45)

print("\n📊 Per class:")
for i, name in metrics.names.items():
    print(f"  {name:<22}: {metrics.box.maps[i]:.4f}")

In [ ]:
import glob, random, cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load model multi-scale (đang có sẵn trong session)
# Nếu model chưa load thì uncomment dòng dưới:
# model = YOLO('/kaggle/working/outputs/yolov12_multiscale_bifpn/weights/best.pt')

test_files = glob.glob('/kaggle/working/yolov12/rice-disease-3/test/images/*.jpg')
if not test_files:
    test_files = glob.glob('/kaggle/working/yolov12/rice-disease-3/test/images/*.png')

print(f"✅ Tìm thấy {len(test_files)} ảnh test")

random.seed(42)  # Seed cố định để reproduce
random_files = random.sample(test_files, min(12, len(test_files)))

results = model.predict(
    source  = random_files,
    conf    = 0.25,
    imgsz   = 640,
    verbose = False,
)

rows, cols = 3, 4
plt.figure(figsize=(20, 15))

for i, result in enumerate(results):
    img_rgb = cv2.cvtColor(result.plot(line_width=2), cv2.COLOR_BGR2RGB)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(random_files[i].split('/')[-1], fontsize=8)

plt.suptitle(
    "YOLOv12 Multi-Scale BiFPN | mAP@50 = 81.5% | mAP@50-95 = 57.2%",
    fontsize=13, fontweight='bold'
)
plt.tight_layout()
plt.savefig('/kaggle/working/yolov12_multiscale_visualization.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Đã lưu visualization!")

In [ ]:
import shutil, os

# Tạo folder zip gọn
os.makedirs('/kaggle/working/download_package', exist_ok=True)

# 1. best.pt - quan trọng nhất!
shutil.copy(
    '/kaggle/working/outputs/yolov12_multiscale_bifpn/weights/best.pt',
    '/kaggle/working/download_package/yolov12_multiscale_best.pt'
)

# 2. results.csv - training history
shutil.copy(
    '/kaggle/working/outputs/yolov12_multiscale_bifpn/results.csv',
    '/kaggle/working/download_package/yolov12_multiscale_results.csv'
)

# 3. Các plots (confusion matrix, PR curve...)
for f in ['confusion_matrix.png', 'results.png', 'PR_curve.png', 'F1_curve.png']:
    src = f'/kaggle/working/outputs/yolov12_multiscale_bifpn/{f}'
    if os.path.exists(src):
        shutil.copy(src, f'/kaggle/working/download_package/{f}')

# Zip lại
shutil.make_archive('/kaggle/working/yolov12_multiscale_package', 'zip', 
                    '/kaggle/working/download_package')

print("✅ Package sẵn sàng!")
print(f"   Size: {os.path.getsize('/kaggle/working/yolov12_multiscale_package.zip')/1024/1024:.1f} MB")

# List những gì trong package
for f in os.listdir('/kaggle/working/download_package'):
    size = os.path.getsize(f'/kaggle/working/download_package/{f}')
    print(f"   📦 {f} - {size/1024/1024:.1f} MB")

In [ ]:
# Chạy cell này để tạo Kaggle dataset metadata
import json, os

os.makedirs('/kaggle/working/model_upload', exist_ok=True)

# Copy best.pt vào folder upload
import shutil
shutil.copy(
    '/kaggle/working/outputs/yolov12_multiscale_bifpn/weights/best.pt',
    '/kaggle/working/model_upload/yolov12_multiscale.pt'
)

# Tạo metadata
metadata = {
    "title": "yolov12-multiscale-bifpn",
    "id": "anhoang5205/yolov12-multiscale-bifpn",
    "licenses": [{"name": "CC0-1.0"}]
}

with open('/kaggle/working/model_upload/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f)

print("✅ Sẵn sàng upload!")
print("Giờ vào: kaggle.com → Datasets → New Dataset → upload folder model_upload/")